<table style="width:99%">
<tr>    
<td style="text-align: center">
<h1>Lab 03: Assignment</h1>
<h2>Arkadiusz Paterak</h2>
</td>
</tr>
</table>

In [ ]:
import pint
from pint import Quantity
import numpy as np
import scipy.constants
import matplotlib.pyplot as plt
from open_atmos_jupyter_utils import show_plot

from typing import Callable